# Modeling Sequential Decision Problems Mathematically

## Compact description

Sequential decision problems are processes of the form:$$ decision, information, decision, information, decision \ldots $$

Any such problem can be modeled as $$ ( S_0, x_0, W_1, S_1, x_1, W_2, \ldots , S_t, x_t, W_{t+1}, \ldots , S_T ), $$

where:
* $S_0$ is the *initial state variable* that captures:
    1. deterministic parameter values that never change
    2. initial values of quantities or parameters that do change
    3. beliefs about quantities or parameters that we don't know perfectly
* $S_t$ for $t\gt0$ are *dynamic state variables* that captures all the information about the state of the system at time $t$ we need to compute the next state. There are three kinds:
    1. Physical state $R_t$
    2. Information state $I_t$
    3. Belief state $B_t$
* $x_t$ are *decision variables* that capture elements that controlled.
* $W_t$ are *exogenous information* that arrives after a decision $x_t$.

Sometimes it's more natural to index decisions using a counter $n$. Then the process is$$ ( S^0, x^0, W^1, S^1, x^1, W^2, \ldots , S^n, x^n, W^{n+1}, \ldots , S^N ). $$And sometimes both index and time are captured, as in $(S^n_t, x^n_t,W^n_t)$.

States are updated by a *transition function* $S^M$ such that$$ S_{t+1} = S^M(S_t,x_t,W_t). $$

Decision variables $x_t$ are determined by a *policy* function denoted $X^{\pi}(S_t)$.

Decisions incur a *cost* or *contribution* $$C(S_t,x_t)=C(S_t,X^{\pi}(S_t)).$$

The objective is to find the best policy to optimize some metric. The most common way of writing the *objective function* is
$$ \max_{\pi} \mathbb{E} \left \{ \sum^T_{t=0} C ( S_t,X^{\pi}(S_t) ) \mid S_0 \right \}. $$

## 6-Step Modeling Process 

Powell describes the modeling process in six steps:

1. Write a big picture **narrative** description of the process in plain English
2. Write down the **core elements** of the problem:
    * The metrics being impacted
    * The decisions being made
    * The sources of uncertainty
3. Write down a **mathematical model** including all the elements in the compact description, including
    * Initial state variable $S_0$
    * Dynamic state variables $S_t$ for $t\gt0$
    * Decision variables $x_t$
    * Exogenous information variables $W_{t+1}$
    * The transition function $S^M(S_t,x_t,W_{t+1})$
    * The cost / contribution function $C(S_t, x_t)$
    * The objective function
4. Write down the **uncertainty model**
    * The initial state $S_0$ might specify distributions of uncertain state transition parameters
    * Specify the exogenous information process through one of three ways:
        1. Create a mathematical model
        2. Use observations of past events
        3. Run field experiments
5. Design **policy functions** using one of two core strategies
    1. Define a discrete set of functions to search over
    2. Define a parametric family of functions and an estimation of of the expected objective value of different parameter values
6. Find the best policy by **evaluating policies** through simulation or experimentation

## A Simple Example: Inventory Problem 

Powell provides the following simple example to illustrate the above ideas.

### 1. Narative

A pizza restaurant has to decide how many pounds of sausage to order from its food distributor. The restaurant has to make the decision at the end of day $t$, communicate the order which then arrives the following morning to meet tomorrow’s orders. If there is sausage left over, it can be held to the following day. The cost of the sausage, and the price that it will be sold for the next day, is known in advance, but the demand is not.

### 2. Core Elements

* **Metrics** - We want to maximize profits given by the sales of sausage minus the cost of purchasing the sausage.
* **Decisions** - We have to decide how much to order at the end of one day. Orders arrive at the beginning of the next day.
* **Sources of Uncertainty** - The only source of uncertainty in this example is the demand for sausage the next day. 

### 3. Mathematical Model

#### State variables

The *initial state variable* is

$$S_0 = (p,c,\bar{D},\bar{\sigma}^D,R_0^{\text{inv}}),$$

where $p$ is the price we charge for sausage, $c$ is our purchase cost of sausage, and $(\bar{D},\bar{\sigma}^D)$ are the mean and standard deviation of the demand, and $R_0^{\text{inv}}$ is the initial inventory level of sausage.

The *dynamic state variable* for $t\gt0$ is

$$S_t=R_t^{\text{inv}},$$

where $R_t^{\text{inv}}$ is the amount of sausage in inventory at time $t$.

#### Decision variable

The decision variable $x_t$ is how much sausage we order at time $t$.

#### Exogenous information

The exogenous information is the random demand for sauage, denoted

$$W_{t+1}=\hat{D}_{t+1}.$$

#### Transition Function

The inventory left at the end of the day $t+1$, will be the inventory left at the end of day $t$, plus how much we order on day $t$, minus the random demand on day $t+1$, or zero if we run out of sausage before the full demand is met. We can write that as follows:

$$ R_{t+1}^{\text{inv}} =\max{\left\{ 0, R_t^{\text{inv}} +x_t + \hat{D}_{t+1} \right\}}. $$

#### Contribution Function

The contribution to profit for buying $x_t$ sausage on day $t$ is the selling price $p$ of sausage multiplied by how much was sold, which is the least of the full inventory $R_t^{\text{inv}} +x_t$ at day $t+1$ or the random demand $\hat{D}_{t+1}$ on day $t+1$, minus the cost of sausage $c$ multiplied by how much was bought $x_t$. The following formula will compute the contribution:

$$ C(S_t,x_t,\hat{D}_{t+1}) = p \min \left \{R^{\text{inv}}_t+x_t,\hat{D}_{t+1} \right \} -cx_t. $$

#### Objective Function

The objective is to find the policy $\pi$ that maximize sum total of profits over all time periods, averaged over the random demand process $\hat{D}_1,\ldots,\hat{D}_T$.

$$ \max_{\pi} F^{\pi} = \max_{\pi} \mathbb{E} \left \{ \sum_{t=0}^T C(S_t,X^{\pi}(S_t),\hat{D}_{t+1}) \mid S_0 \right \}. $$

### 4. Uncertainty Model

A simple mathematical model for the demand process is to assume demand is normally distributed with some mean $\bar{D}$ and standard deviation $\bar{\sigma}^D$. If these are known, demand can be written as

$$ \hat{D}_{t+1} \sim N (\bar{D}, (\bar{\sigma}^D)^2 ). $$

### 5. Policies

A common policy for inventory problems is called "order-up-to" which can be written as

$$ 
X^{\pi}(S_t \mid \theta)=\bigg\{\begin{array}{cl}
\theta^{max} - R_t & \text{if } R_t \lt \theta^{min}, \\
0 & \text{otherwise} ,
\end{array}
$$

where $\theta=(\theta^{min},\theta^{max})$ is a set of tunable parameters that define a lower and upper limit for inventory levels such that inventories are restored up to the upper limit when they drop below the lower limit.

### 6. Evaluation

Even though there is no analytical way to compute the value of the objective function exactly, we can resort to numerical estimation using simulation. We can simulate $N$ sample paths of random demands $\hat{D}^n_1,\ldots,\hat{D}^n_T$ and estimate the expected profits by averaging over these samples like this:

$$ \bar{F}^{\pi}(\theta) = \frac{1}{N} \sum_{n=1}^{N} \sum_{t=0}^{T} C(S_t,X^{\pi}(S_t\mid\theta),\hat{D}^n_{t+1}). $$

Then we're left to find the value of $\theta=(\theta^{min},\theta^{max})$ that gives the highest average profit as computed above.